In [1]:
DATA_ROOT = 'images'

In [2]:
from collections import defaultdict, Counter
import random

import pandas as pd
import tqdm

You can also use ready-made already folded folds.


In [3]:
folds = pd.read_csv('folds.csv')

In [4]:
folds.head(5)

,image_id,target,fold
0,Train_0,3,3
1,Train_1,1,1
2,Train_2,0,0
3,Train_3,2,1
4,Train_4,0,1


In [5]:
from pathlib import Path
from typing import Callable, List

import cv2
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset

In [6]:
N_CLASSES = 4

In [7]:
class TrainDataset(Dataset):
    def __init__(self, root: Path, df: pd.DataFrame,
                 image_transform: Callable, debug: bool = True):
        super().__init__()
        self._root = root
        self._df = df
        self._image_transform = image_transform
        self._debug = debug

    def __len__(self):
        return len(self._df)

    def __getitem__(self, idx: int):
        item = self._df.iloc[idx]
        image = load_transform_image(
            item, self._root, self._image_transform, debug=self._debug)
        target = torch.tensor(item.target)
        return image, target


class TTADataset:
    def __init__(self, root: Path, df: pd.DataFrame,
                 image_transform: Callable, tta: int):
        self._root = root
        self._df = df
        self._image_transform = image_transform
        self._tta = tta

    def __len__(self):
        return len(self._df) * self._tta

    def __getitem__(self, idx):
        item = self._df.iloc[idx % len(self._df)]
        image = load_transform_image(item, self._root, self._image_transform)
        return image, item.image_id

## Transforms

In [8]:
import random
import math

from PIL import Image
from torchvision.transforms import (
    ToTensor, Normalize, Compose, Resize, CenterCrop, RandomCrop,
    RandomHorizontalFlip)

In [9]:
train_transform = Compose([
    RandomCrop(288),
    RandomHorizontalFlip(),
])

test_transform = Compose([
    #RandomCrop(288),
    RandomCrop(256),
    RandomHorizontalFlip(),
])

tensor_transform = Compose([
    ToTensor(),
    Normalize(mean=[0.496,0.456,0.406], std=[0.229, 0.224, 0.225]),
])

In [10]:
def load_transform_image(
        item, root: Path, image_transform: Callable, debug: bool = False):
    image = load_image(item, root)
    image = image_transform(image)
    if debug:
        image.save('_debug.png')
    return tensor_transform(image)


def load_image(item, root: Path) -> Image.Image:
    #print(str(root + '/' + f'{item.image_id}.jpg'))
    image = cv2.imread(str(root + '/' + f'{item.image_id}.jpg'))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return Image.fromarray(image)


def get_ids(root: Path) -> List[str]:
    return sorted({p.name.split('_')[0] for p in root.glob('*.jpg')})

## Load data 

In [11]:
folds = pd.read_csv('folds.csv')

In [12]:
fold = 0

In [13]:
train_fold = folds[folds['fold'] != 0]
valid_fold = folds[folds['fold'] == 0]

In [14]:
from itertools import islice
import json
from pathlib import Path
import shutil
import warnings
from typing import Dict

import numpy as np
import pandas as pd
from sklearn.metrics import fbeta_score, roc_auc_score
from sklearn.exceptions import UndefinedMetricWarning
import torch
from torch import nn, cuda
from torch.optim import Adam
import tqdm

In [15]:
from torch.utils.data import DataLoader

In [16]:
train_root = DATA_ROOT #+ 'train'

In [17]:
num_workers = 4
batch_size = 64

In [18]:
def make_loader(df: pd.DataFrame, image_transform) -> DataLoader:
        return DataLoader(
            TrainDataset(train_root, df, image_transform, debug=0),
            shuffle=True,
            batch_size=batch_size,
            num_workers=num_workers,
        )

In [19]:
train_loader = make_loader(train_fold, train_transform)
valid_loader = make_loader(valid_fold, test_transform)
print(f'{len(train_loader.dataset):,} items in train, '
      f'{len(valid_loader.dataset):,} in valid')

1,434 items in train, 387 in valid


## Models

In [20]:
from torch.nn import functional as F
import torchvision.models as M
from functools import partial

In [21]:
class DenseCrossEntropy(nn.Module):

    def __init__(self):
        super(DenseCrossEntropy, self).__init__()
        
        
    def forward(self, logits, labels):
        logits = logits.float()
        labels = labels.float()
        
        logprobs = F.log_softmax(logits, dim=-1)
        
        loss = -labels * logprobs
        loss = loss.sum(-1)

        return loss.mean()

In [22]:
class AvgPool(nn.Module):
    def forward(self, x):
        return F.avg_pool2d(x, x.shape[2:])


def create_net(net_cls, pretrained: bool):
    if pretrained:
        net = net_cls()
        model_name = net_cls.__name__
        weights_path = f'../input/{model_name}/{model_name}.pth'
        net.load_state_dict(torch.load(weights_path))
    else:
        net = net_cls(pretrained=pretrained)
    return net


class ResNet(nn.Module):
    def __init__(self, num_classes,
                 pretrained=False, net_cls=M.resnet50, dropout=False):
        super().__init__()
        self.net = create_net(net_cls, pretrained=pretrained)
        self.net.avgpool = AvgPool()
        if dropout:
            self.net.fc = nn.Sequential(
                nn.Dropout(),
                nn.Linear(self.net.fc.in_features, num_classes),
            )
        else:
            self.net.fc = nn.Linear(self.net.fc.in_features, num_classes)

    def fresh_params(self):
        return self.net.fc.parameters()

    def forward(self, x):
        return self.net(x)


class DenseNet(nn.Module):
    def __init__(self, num_classes,
                 pretrained=False, net_cls=M.densenet121):
        super().__init__()
        self.net = create_net(net_cls, pretrained=pretrained)
        self.avg_pool = AvgPool()
        self.net.classifier = nn.Linear(
            self.net.classifier.in_features, num_classes)

    def fresh_params(self):
        return self.net.classifier.parameters()

    def forward(self, x):
        out = self.net.features(x)
        out = F.relu(out, inplace=True)
        out = self.avg_pool(out).view(out.size(0), -1)
        out = self.net.classifier(out)
        return out


resnet18 = partial(ResNet, net_cls=M.resnet18)
resnet34 = partial(ResNet, net_cls=M.resnet34)
resnet50 = partial(ResNet, net_cls=M.resnet50)
resnet101 = partial(ResNet, net_cls=M.resnet101)
resnet152 = partial(ResNet, net_cls=M.resnet152)

densenet121 = partial(DenseNet, net_cls=M.densenet121)
densenet169 = partial(DenseNet, net_cls=M.densenet169)
densenet201 = partial(DenseNet, net_cls=M.densenet201)
densenet161 = partial(DenseNet, net_cls=M.densenet161)

In [23]:
criterion = nn.CrossEntropyLoss() #DenseCrossEntropy()#nn.BCEWithLogitsLoss(reduction='none')

In [24]:
use_cuda = cuda.is_available()

In [25]:
model = resnet50(num_classes=N_CLASSES, pretrained=False)

In [26]:
fresh_params = list(model.fresh_params())
all_params = list(model.parameters())
if use_cuda:
    model = model.cuda()

In [27]:
 train_kwargs = dict(
            model=model,
            criterion=criterion,
            train_loader=train_loader,
            valid_loader=valid_loader,
            patience=4,
            init_optimizer=lambda params, lr: Adam(params, lr),
            use_cuda=use_cuda,
        )

In [28]:
def load_model(model: nn.Module, path: Path) -> Dict:
    state = torch.load(str(path))
    model.load_state_dict(state['model'])
    print('Loaded model from epoch {epoch}, step {step:,}'.format(**state))
    return state

In [29]:
def _reduce_loss(loss):
    print(loss)
    return loss.sum() / loss.shape[0]

In [30]:
def binarize_prediction(probabilities, threshold: float, argsorted=None,
                        min_labels=1, max_labels=10):
    """ Return matrix of 0/1 predictions, same shape as probabilities.
    """
    assert probabilities.shape[1] == N_CLASSES
    if argsorted is None:
        argsorted = probabilities.argsort(axis=1)
    max_mask = _make_mask(argsorted, max_labels)
    min_mask = _make_mask(argsorted, min_labels)
    prob_mask = probabilities > threshold
    return (max_mask & prob_mask) | min_mask


def _make_mask(argsorted, top_n: int):
    mask = np.zeros_like(argsorted, dtype=np.uint8)
    col_indices = argsorted[:, -top_n:].reshape(-1)
    row_indices = [i // top_n for i in range(len(col_indices))]
    mask[row_indices, col_indices] = 1
    return mask


In [31]:
def validation(
        model: nn.Module, criterion, valid_loader, use_cuda,
        ) -> Dict[str, float]:
    model.eval()
    val_loss = 0
    val_preds = None
    val_labels = None
    for inputs, targets in valid_loader:
        images = inputs
        labels = targets

        if val_labels is None:
            val_labels = labels.clone().squeeze(-1)
        else:
            val_labels = torch.cat((val_labels, labels.squeeze(-1)), dim=0)

        images = images.cuda()
        labels = labels.cuda()

        with torch.no_grad():
            outputs = model(images)

            loss = criterion(outputs, labels.squeeze(-1))
            val_loss += loss.item()

            preds = torch.nn.functional.softmax(model(images), 1)
            #print(preds)

            if val_preds is None:
                val_preds = preds
            else:
                val_preds = torch.cat((val_preds, preds), dim=0)
            
    metrics = {}
    metrics['valid_loss'] = np.mean(val_loss)
    metrics['roc_auc'] = roc_auc_score(val_labels, val_preds.cpu(), multi_class="ovr",average='weighted')
    print(' | '.join(f'{k} {v:.3f}' for k, v in sorted(metrics.items(), key=lambda kv: -kv[1])))
    return metrics
#     all_losses, all_predictions, all_targets = [], [], []
#     with torch.no_grad():
#         for inputs, targets in valid_loader:
#             all_targets.append(targets.numpy().copy())
#             if use_cuda:
#                 inputs, targets = inputs.cuda(), targets.cuda()
#             outputs = model(inputs)
#             loss = criterion(outputs, targets)
#             #all_losses.append(_reduce_loss(loss).item())
#             all_losses.append(loss)
#             #print(outputs)
#             predictions = torch.softmax(outputs, dim=1).data.cpu()#torch.argmax(outputs)
#             all_predictions.append(predictions.cpu().numpy())
#     all_predictions = np.concatenate(all_predictions)
#     all_targets = np.concatenate(all_targets)

#     def get_score(y_pred):
#         print(all_targets)
#         with warnings.catch_warnings():
#             warnings.simplefilter('ignore', category=UndefinedMetricWarning)
#             return roc_auc_score(
#                 all_targets, y_pred)#, beta=2, average='samples')

#     metrics = {}
#     argsorted = all_predictions.argsort(axis=1)
# #     for threshold in [0.10, 0.20]:
# #         metrics[f'valid_f2_th_{threshold:.2f}'] = get_score(
# #             binarize_prediction(all_predictions, threshold, argsorted))
# #     metrics['valid_loss'] = np.mean(all_losses)
#     print(all_predictions)
#     get_score(all_predictions)
#     print(' | '.join(f'{k} {v:.3f}' for k, v in sorted(
#         metrics.items(), key=lambda kv: -kv[1])))

#     return metrics

In [32]:
def train( model: nn.Module, criterion, *, params,
          train_loader, valid_loader, init_optimizer, use_cuda,
          n_epochs=None, patience=2, max_lr_changes=2) -> bool:
    
    lr = 1e-4
    batch_size = 32
    n_epochs = 40
    params = list(params)
    optimizer = init_optimizer(params, lr)

    model_path = 'model.pt'
    best_model_path = 'best-model.pt'
    uptrain = False
    if uptrain:
        state = load_model(model, model_path)
        epoch = state['epoch']
        step = state['step']
        best_valid_loss = state['best_valid_loss']
    else:
        epoch = 1
        step = 0
        best_valid_loss = float('inf')
    lr_changes = 0

    save = lambda ep: torch.save({
        'model': model.state_dict(),
        'epoch': ep,
        'step': step,
        'best_valid_loss': best_valid_loss
    }, str(model_path))

    report_each = 100
    valid_losses = []
    lr_reset_epoch = epoch
    for epoch in range(epoch, n_epochs + 1):
        model.train()
        tq = tqdm.tqdm(total=(len(train_loader) * batch_size))
        tq.set_description(f'Epoch {epoch}, lr {lr}')
        losses = []
        tl = train_loader
        try:
            mean_loss = 0
            for i, (inputs, targets) in enumerate(tl):
                if use_cuda:
                    inputs, targets = inputs.cuda(), targets.cuda()
                outputs = model(inputs)
                #print(outputs, targets)
                #loss = _reduce_loss(criterion(outputs, targets))
                loss = criterion(outputs, targets)
                batch_size = inputs.size(0)
                #(batch_size * loss).backward()
                loss.backward()
                if (i + 1) % 1 == 0:
                    optimizer.step()
                    optimizer.zero_grad()
                    step += 1
                tq.update(batch_size)
                #print(loss.item())
                #losses.append(loss.item())
                losses.append(loss)
                #print(len(losses))
                mean_loss = losses[0]# np.mean(losses[-report_each:])
                tq.set_postfix(loss=f'{mean_loss:.3f}')
            tq.close()
            save(epoch + 1)
            valid_metrics = validation(model, criterion, valid_loader, use_cuda)
            
            valid_loss = valid_metrics['valid_loss']
            valid_losses.append(valid_loss)
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                shutil.copy(str(model_path), str(best_model_path))
            elif (patience and epoch - lr_reset_epoch > patience and
                  min(valid_losses[-patience:]) > best_valid_loss):
                lr_changes +=1
                if lr_changes > max_lr_changes:
                    break
                lr /= 5
                print(f'lr updated to {lr}')
                lr_reset_epoch = epoch
                optimizer = init_optimizer(params, lr)
        except KeyboardInterrupt:
            tq.close()
            print('Ctrl+C, saving snapshot')
            save(epoch)
            print('done.')
            return False
    return True

In [33]:
train(params=all_params, **train_kwargs)

Epoch 1, lr 0.0001: : 1434it [00:12, 116.27it/s, loss=1.650]
Epoch 2, lr 0.0001:   0%|          | 0/598 [00:00<?, ?it/s]

valid_loss 9.027 | roc_auc 0.568


Epoch 2, lr 0.0001: : 1434it [00:12, 111.78it/s, loss=1.289]
Epoch 3, lr 0.0001:   0%|          | 0/598 [00:00<?, ?it/s]

valid_loss 9.453 | roc_auc 0.584


Epoch 3, lr 0.0001: : 1434it [00:12, 112.51it/s, loss=1.176]
Epoch 4, lr 0.0001:   0%|          | 0/598 [00:00<?, ?it/s]

...

Epoch 30, lr 4.000000000000001e-06: : 1434it [00:12, 110.89it/s, loss=1.097]
Epoch 31, lr 4.000000000000001e-06:   0%|          | 0/598 [00:00<?, ?it/s]

valid_loss 8.116 | roc_auc 0.704


Epoch 31, lr 4.000000000000001e-06: : 1434it [00:13, 110.21it/s, loss=0.993]


valid_loss 7.615 | roc_auc 0.717


True